In [1]:
import nglview as nv

import json
from pymatgen.core import Lattice, Structure, Molecule

In [2]:
def read_pymatgen_dict(file):
    with open(file, "r") as f:
        d = json.load(f)
    return Structure.from_dict(d)

In [3]:
import os

filepath_train = '../Data/dichalcogenides_public/structures/'
train_files = os.listdir(filepath_train)

filepath_test = '../Data/dichalcogenides_private/structures/'
test_files = os.listdir(filepath_test)

In [4]:
file = test_files[3]
crystal = read_pymatgen_dict(filepath_test+file)

In [5]:
file = train_files[0]
crystal = read_pymatgen_dict(filepath_train+file)

In [6]:
view = nv.show_pymatgen(crystal)
view.add_unitcell()
view

NGLWidget()

In [910]:
f1 = '6141eb094e27a1844a5f03dc'
f2 = '6141cf1202d926221cabc545'

crystal1 = read_pymatgen_dict(filepath_train+f1+'.json')
crystal2 = read_pymatgen_dict(filepath_train+f2+'.json')

In [911]:
view = nv.show_pymatgen(crystal1)
view.add_unitcell()
view

NGLWidget()

In [907]:
view = nv.show_pymatgen(crystal2)
view.add_unitcell()
view

NGLWidget()

In [892]:
import pandas as pd
import numpy as np

In [912]:
cr1 = pd.DataFrame(crystal1.as_dict()['sites'])
cr1abc = pd.DataFrame(list(cr1['abc']))
cr1abc_c = cr1abc.copy()

In [913]:
def rotate120(x,y):
    a = 120
    a = a / 180 * np.pi
    c, s = np.cos(a), np.sin(a)
    B = np.array([[1, c],[0, s]])
    Binv = np.linalg.inv(B)
    R = np.array([[c,-s],[s,c]])
    v = np.array([x,y])
    v = Binv @ R @ B @ v
    return -y, x-y

In [914]:
def togrid(x,y,z):
    xy = cr1abc[cr1abc[2]==z]
    if (z != 0.25):
        y = np.round(y, 7)
    xx = xy[0]
    yy = xy[1]
    
    xm, xM = xx.min(), xx.max()
    ym, yM = yy.min(), yy.max()
    
    x = (x - xm) % ((xM - xm) / 7 * 8) + xm
    y = (y - ym) % ((yM - ym) / 7 * 8) + ym
    return np.round(x, 8), np.round(y, 8)

In [915]:
def rot_mapping(row):
    x, y, z = row
    x, y = rotate120(x, y)
    x, y = togrid(x, y, z)
    return [x, y, z]

In [916]:
def translate(x, y, z, vx, vy, vz):
    return x + vx, y + vy, z + vz

def trans_mapping(row, vx, vy, vz):
    x, y, z = row
    x, y, zz = translate(x, y, z, vx, vy, vz)
    x, y = togrid(x, y, z)
    return [x, y, zz]

In [848]:
cr1abc = cr1abc_c.apply(rot_mapping, axis=1, result_type='expand')
cr1abc_c = cr1abc.copy()
cr1abc = cr1abc_c.apply(rot_mapping, axis=1, result_type='expand')

In [922]:
cr1abc_c = cr1abc.copy()
cr1abc = cr1abc_c.apply(trans_mapping, axis=1, result_type='expand', args=(0.2, 0, 0))

In [923]:
cr1['abc'] = pd.Series(cr1abc.to_numpy().tolist(), index=cr1.index)

In [924]:
crd = crystal1.as_dict()
crd['sites'] = [cr1.iloc[i].to_dict() for i in range(len(cr1))]

In [925]:
crystal_new = Structure.from_dict(crd)

In [926]:
view = nv.show_pymatgen(crystal_new)
view.add_unitcell()
view

NGLWidget()

In [902]:
view = nv.show_pymatgen(crystal2)
view.add_unitcell()
view

NGLWidget()

In [5]:
nv.write_html('embed.html', view)

In [6]:
coords = [[0, 0, 0], [0.75,0.5,0.75]]
lattice = Lattice.from_parameters(a=3.84, b=3.84, c=3.84, alpha=120,
                                  beta=90, gamma=60)
struct = Structure(lattice, ["Si", "Si"], coords)

coords = [[0.000000, 0.000000, 0.000000],
          [0.000000, 0.000000, 1.089000],
          [1.026719, 0.000000, -0.363000],
          [-0.513360, -0.889165, -0.363000],
          [-0.513360, 0.889165, -0.363000]]
methane = Molecule(["C", "H", "H", "H", "H"], coords)

nv.show_pymatgen(methane)

NGLWidget()